In [16]:
from sklearn.base import BaseEstimator, ClassifierMixin, clone
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

In [4]:
a = 0.1*np.ones(10,dtype=float)
a

array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])

In [61]:
class SimpleMultiClassBoosting(BaseEstimator, ClassifierMixin):
    def __init__(self, base_estimator=None, n_estimators=50, weight_increase_ratio = 0.5):
        self.base_estimator = base_estimator if base_estimator is not None else DecisionTreeClassifier(max_depth=1)
        self.n_estimators = n_estimators
        self.learners = np.empty(n_estimators, dtype=object)
        self.learner_weights = np.empty(n_estimators, dtype=float)
        self.samples_weights = None
        self.weigh_increase_ratio = weight_increase_ratio
        self.label_encoder = LabelEncoder()

    def fit(self, X, y):
        # Convert labels to [0, n_classes-1]
        Y = self.label_encoder.fit_transform(y=y)
        n_classes = np.unique(Y)
        indexes = np.arange(X.shape[0])
        # Initialize weights uniformly
        self.samples_weights = (1/X.shape[0]) * np.ones(X.shape[0] ,dtype=float)
        for i in range(self.n_estimators):
            self.learners[i] = self.base_estimator
            training_indexes = np.random.choice(indexes, indexes.shape[0], replace=True, p=self.samples_weights)
            self.learners[i].fit(X[training_indexes], Y[training_indexes])
            # Compute weighted error rate (misclassification rate)
            missClassifiedIndexes = []
            predictions = self.predict(X=X)
            print(predictions)
            for i in range(X.shape[0]):
                if predictions[i] != y[i]:
                    missClassifiedIndexes.append(i)
            learner_error = (np.sum(self.samples_weights[missClassifiedIndexes])) / np.sum(self.samples_weights)        

            # Compute learner weight using SAMME algorithm
            self.learner_weights[i] = 0.5*np.log((1 - learner_error) / learner_error)
            if learner_error >= 1 - (1 / n_classes):
                self.learner_weights[i] = 0

            # Increase the weights of misclassified samples
            for index, weigth in enumerate(self.samples_weights):
                if index in missClassifiedIndexes:
                    self.samples_weights[index] += self.weigh_increase_ratio

            self.samples_weights = self.samples_weights / (np.sum(self.samples_weights))        
            # Save the current learner
            # self.learners.append(learner)
            # self.learner_weights.append(learner_weight)

    def predict(self, X):
        # Collect predictions from each learner
        predictions = []
        for learner in self.learners:
            if learner != None:
                predictions.append(learner.predict(X))
        print(predictions.shape)
        # labels = np.unique(predictions)
        # votes = {label : 0 for label in labels}
        # for i in range(len(predictions)):
        #     for label in labels:
        #         if predictions[0][i] == label:
        #             votes[label] += self.learner_weights[i]
        
        # # Weighted vote for each sample's prediction across all learners
        # final_prediction = max(votes, key=votes.get)
        # Final prediction is the one with the highest weighted vote
        return self.label_encoder.inverse_transform(np.array([final_prediction]))
        # Convert back to original class labels


In [23]:
iris = load_iris()
df = pd.DataFrame(data=np.c_[iris['data'], iris['target']], columns=iris['feature_names'] + ['target'])
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [51]:
X_train, X_test, Y_train, Y_test = train_test_split(df.iloc[:, :-1].to_numpy(), df.iloc[:, -1].to_numpy(), test_size=0.2, random_state=42)

In [62]:
m = SimpleMultiClassBoosting()
m.fit(X_train, Y_train)

AttributeError: 'list' object has no attribute 'shape'